In [0]:
from langchain.tools import tool
import json
import pandas as pd
from langchain_tavily import TavilySearch
from langchain_community.tools import DuckDuckGoSearchRun

In [0]:
# Ferramenta 1: Consultar métricas do SRAG
@tool
def consultar_metricas_srag() -> str:
    '''Consulta métricas pré-calculadas, gera gráficos de casos recentes e retorna tudo em JSON (incluindo base64 dos gráficos).'''
    
    try:
        df_metrics = spark.table("workspace.default.srag_metrics").toPandas()
        
        data_ref = df_metrics['data_referencia'].iloc[0]
        data_ref = pd.to_datetime(data_ref) 
        
        metricas = {
            "taxa_aumento_casos_7d": f"{df_metrics[df_metrics['metrica']=='taxa_aumento_casos_7d']['valor'].iloc[0]:.2f}%",
            "taxa_mortalidade": f"{df_metrics[df_metrics['metrica']=='taxa_mortalidade']['valor'].iloc[0]:.2f}%",
            "taxa_uti_proxy": f"{df_metrics[df_metrics['metrica']=='taxa_uti_proxy']['valor'].iloc[0]:.2f}%",
            "taxa_vacinacao_proxy": f"{df_metrics[df_metrics['metrica']=='taxa_vacinacao_proxy']['valor'].iloc[0]:.2f}%"
        }
        
        return json.dumps({
            "data_referencia": str(data_ref),
            "metricas": metricas
        }, ensure_ascii=False)
        
    except Exception as e:
        return json.dumps({"erro": str(e)}, ensure_ascii=False)

In [0]:
@tool
def buscar_noticias_srag() -> str:
    '''Busca notícias recentes sobre SRAG no Brasil para embasar o relatório.
    Usa busca real em fontes confiáveis'''
    query = f'SRAG Brasil notícias recentes 2025'

    search = TavilySearch(
        max_results=5,
        search_depth="advanced",
        include_domains=["fiocruz.br", "saude.gov.br", "g1.globo.com", "folha.uol.com.br"]
    )
    
    resultados = search.run(query)
    return resultados    
    
tools = [consultar_metricas_srag, buscar_noticias_srag]

In [0]:
import os

os.environ["TAVILY_API_KEY"] = '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'

buscar_noticias_srag.invoke(None)

In [0]:
consultar_metricas_srag.invoke(None)